In [88]:
!pip install -U langchain langchain-mcp-adapters langchain-openai langgraph elasticsearch dotenv rich -q


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [89]:
import json
import os

from typing import TypedDict

import requests
from dotenv import load_dotenv

from elasticsearch import Elasticsearch, helpers

from langchain.agents import create_agent
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.graph import END, START, StateGraph
from pydantic import BaseModel, Field

from rich.console import Console
from rich.markdown import Markdown
from rich.panel import Panel

console = Console()

In [90]:
load_dotenv()

ELASTICSEARCH_ENDPOINT = os.getenv("ELASTICSEARCH_ENDPOINT")
ELASTICSEARCH_API_KEY = os.getenv("ELASTICSEARCH_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
KIBANA_URL = os.getenv("KIBANA_URL")

INDEX_NAME = "security-vulnerabilities"
KIBANA_HEADERS = {
    "kbn-xsrf": "true",
    "Content-Type": "application/json",
    "Authorization": f"ApiKey {ELASTICSEARCH_API_KEY}",
}  # Useful for Agent Builder setup API calls

MCP_ENDPOINT = f"{KIBANA_URL}/api/agent_builder/mcp"

In [91]:
es_client = Elasticsearch(ELASTICSEARCH_ENDPOINT, api_key=ELASTICSEARCH_API_KEY)

In [92]:
security_search_tool = {
    "id": "security-semantic-search",
    "type": "index_search",
    "description": "Search internal security documents including incident reports, pentests, internal CVEs, security guidelines, and architecture decisions. Uses semantic search powered by ELSER to find relevant security information even without exact keyword matches. Returns documents with severity assessment and affected systems.",
    "tags": ["security", "semantic", "vulnerabilities"],
    "configuration": {
        "pattern": INDEX_NAME,
    },
}

try:
    response = requests.post(
        f"{KIBANA_URL}/api/agent_builder/tools",
        headers=KIBANA_HEADERS,
        json=security_search_tool,
    )

    if response.status_code == 200:
        print("✅ Security semantic search tool created successfully")
    else:
        print(f"Response: {response.text}")
except Exception as e:
    print(f"❌ Error creating tool: {e}")

Response: {"statusCode":400,"error":"Bad Request","message":"Tool with id security-semantic-search already exists","attributes":{}}


In [93]:
index_mapping = {
    "mappings": {
        "properties": {
            "title": {"type": "text", "copy_to": "semantic_field"},
            "content": {"type": "text", "copy_to": "semantic_field"},
            "doc_type": {"type": "keyword", "copy_to": "semantic_field"},
            "severity": {"type": "keyword", "copy_to": "semantic_field"},
            "affected_systems": {"type": "keyword", "copy_to": "semantic_field"},
            "date": {"type": "date"},
            "semantic_field": {"type": "semantic_text"},
        }
    }
}

if es_client.indices.exists(index=INDEX_NAME) is False:
    es_client.indices.create(index=INDEX_NAME, body=index_mapping)
    print(f"✅ Index '{INDEX_NAME}' created with semantic_text field for ELSER")
else:
    print(f"ℹ️  Index '{INDEX_NAME}' already exists, skipping creation")

ℹ️  Index 'security-vulnerabilities' already exists, skipping creation


In [94]:
def build_bulk_actions(documents, index_name):
    for doc in documents:
        yield {"_index": index_name, "_source": doc}


try:
    with open("dataset.json", "r") as f:
        security_documents = json.load(f)

    success, failed = helpers.bulk(
        es_client,
        build_bulk_actions(security_documents, INDEX_NAME),
        refresh=True,
    )
    print(f"📥 {success} documents indexed successfully")

except Exception as e:
    print(f"❌ Error during bulk indexing: {str(e)}")

📥 6 documents indexed successfully


In [95]:
client = MultiServerMCPClient(
    {
        "agent-builder": {
            "transport": "streamable_http",
            "url": MCP_ENDPOINT,
            "headers": {"Authorization": f"ApiKey {ELASTICSEARCH_API_KEY}"},
        }
    }
)

tools = await client.get_tools()

print(f"📋 MCP Tools available: {[t.name for t in tools]}")

📋 MCP Tools available: ['platform_core_search', 'platform_core_get_document_by_id', 'platform_core_execute_esql', 'platform_core_generate_esql', 'platform_core_get_index_mapping', 'platform_core_list_indices', 'platform_core_index_explorer', 'security-semantic-search']


In [ ]:
llm = ChatOpenAI(model="gpt-4o", openai_api_key=OPENAI_API_KEY)

agent = create_agent(
    llm,
    tools=tools,
    system_prompt="""You are a cybersecurity expert specializing in infrastructure security.

        Your role is to:
        1. Analyze security queries from users
        2. Search internal security documents (incidents, pentests, CVEs, guidelines)
        3. Provide actionable security recommendations
        4. Assess vulnerability severity and impact

        When responding:
        - Always search internal documents first using the agent builder tools
        - Provide specific, technical, and actionable advice
        - Cite relevant internal incidents and documentation
        - Assess severity (critical, high, medium, low)
        - Recommend immediate mitigation steps

        Be concise but comprehensive. Focus on practical security guidance.""",
)

In [97]:
# Define agent state
class AgentState(TypedDict):
    query: str
    agent_builder_response: dict
    external_results: list
    final_response: str
    needs_external_search: bool

In [98]:
@tool("internet_search_tool")
def internet_search_tool(query: str, top_k: int = 3) -> list:
    """
    Search external sources using Serper API for CVE, advisories, and security intelligence.
    """

    url = "https://google.serper.dev/search"
    payload = {"q": query + " CVE security vulnerability", "num": top_k}
    headers_serper = {"X-API-KEY": SERPER_API_KEY, "Content-Type": "application/json"}

    try:
        response = requests.post(url, json=payload, headers=headers_serper)
        response.raise_for_status()
        data = response.json()

        results = []
        for item in data.get("organic", [])[:top_k]:
            results.append(
                {
                    "title": item.get("title"),
                    "snippet": item.get("snippet"),
                    "link": item.get("link"),
                    "source": "Web Search",
                }
            )
        return results
    except Exception as e:
        print(f"⚠️  Error in Serper API: {e}")
        return []

In [ ]:
# Node 1: Call Agent Builder semantic search tool via MCP
async def search_internal_docs(state: AgentState) -> AgentState:
    query = state["query"]
    console.print(f"🔍 [cyan]Searching internal docs for:[/cyan] {query}")

    response = await agent.ainvoke({"messages": [{"role": "user", "content": query}]})

    messages = response.get("messages", [])
    final_message = messages[-1] if messages else None

    if final_message:
        agent_response_text = final_message.content
        console.print(
            f"✅ [green]Agent response:[/green] {agent_response_text[:200]}..."
        )

        state["agent_builder_response"] = {
            "response": {"message": agent_response_text},
            "raw_messages": messages,
        }
    else:
        console.print("⚠️  [yellow]No response from agent[/yellow]")
        state["agent_builder_response"] = {}

    return state


# Node 2: Decide if external search needed
async def decide_external_search(state: AgentState) -> AgentState:
    """
    Decide if we need additional internet search based on Agent Builder results.
    """
    # Extract content from Agent Builder response
    ab_response = state["agent_builder_response"]
    agent_response = (
        ab_response.get("response", {}).get("message", "") if ab_response else ""
    )

    prompt = f"""
      User query: "{state['query']}"
      
      Elasticsearch MCP server found the following internal information:
      {agent_response[:500] if agent_response else 'No response'}...
      
      Should we search external sources (CVE databases, security advisories) for additional context?
      Consider:
      - Search external if query asks about specific CVE numbers or public vulnerabilities
      - Search external if internal findings mention vulnerabilities that might have public advisories
      - Don't search external if internal findings are comprehensive and specific to company systems
    """

    # Structured output for decision making
    class SearchDecision(BaseModel):
        """Decision on whether additional internet search is needed."""

        needs_external_search: bool = Field(
            description="Whether we need to search external CVE databases and security advisories to complement internal findings"
        )
        reasoning: str = Field(description="Brief explanation of the decision")

    llm_with_structure = llm.with_structured_output(SearchDecision)

    decision: SearchDecision = llm_with_structure.invoke(prompt)
    state["needs_external_search"] = decision.needs_external_search

    status = (
        "✅ [green]Yes[/green]"
        if state["needs_external_search"]
        else "❌ [red]No[/red]"
    )
    console.print(f"🤔 [bold]External search needed:[/bold] {status}")
    console.print(f"   [dim]{decision.reasoning}[/dim]")

    return state


# Node 3: External search
async def perform_external_search(state: AgentState) -> AgentState:
    if not state["needs_external_search"]:
        console.print("ℹ️  [blue]External search not needed, skipping.[/blue]")
        state["external_results"] = []
        return state

    query = state["query"]
    console.print(f"🌐 [cyan]Performing external internet search for:[/cyan] {query}")

    results = internet_search_tool.invoke(query)
    state["external_results"] = results

    console.print(f"🔎 [green]External search returned {len(results)} results.[/green]")
    for i, res in enumerate(results):
        console.print(f"  [{i}] [link]{res['link']}[/link] - {res['title']}")

    return state


# Node 4: Generate final response
async def generate_final_response(state: AgentState) -> AgentState:
    """
    Correlate Agent Builder findings with external search and generate comprehensive response.
    """
    console.print("🧠 [magenta]Generating final correlated response...[/magenta]")

    ab_response = state["agent_builder_response"]
    internal_context = (
        ab_response.get("response", {}).get("message", "No internal findings")
        if ab_response
        else "No internal findings"
    )

    # Format external results with links
    external_results_formatted = []
    if state["external_results"]:
        for i, r in enumerate(state["external_results"], 1):
            external_results_formatted.append(
                f"[{i}] {r['title']}\n    Snippet: {r['snippet']}\n    Link: {r['link']}"
            )

    external_context = (
        "\n\n".join(external_results_formatted)
        if external_results_formatted
        else "No external information searched."
    )

    prompt = f"""
    You are a cybersecurity expert. Provide a comprehensive security analysis.

    **User query:** {state['query']}

    **Internal company findings (from internal Elasticsearch knowledge base):**
    {internal_context}

    **External security intelligence (from web sources):**
    {external_context}

    **Your task:**
    1. Correlate internal and external information
    2. Assess vulnerability severity and impact on company systems
    3. Provide specific, actionable mitigation steps
    4. Highlight any gaps in coverage or additional concerns

    **IMPORTANT - Citation Format:**
    - When referencing information from internal Elasticsearch, use: [internal knowledge]
    - When referencing information from external web sources, cite the link number like: [1], [2], etc.
    - Example: "Express 4.17 has a prototype pollution vulnerability [internal knowledge] which is documented as CVE-2022-24999 [1]"

    Be concise, technical, and actionable.
    """

    response = llm.invoke(prompt).content

    # Append external links section at the end if there are external results
    if state["external_results"]:
        response += "\n\n---\n## External References\n"
        for i, r in enumerate(state["external_results"], 1):
            response += f"[{i}] {r['title']}\n    URL: {r['link']}\n\n"

    state["final_response"] = response

    return state

In [100]:
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("search_internal_docs", search_internal_docs)
workflow.add_node("decide_external", decide_external_search)
workflow.add_node("perform_external_search", perform_external_search)
workflow.add_node("generate_response", generate_final_response)

workflow.add_edge(START, "search_internal_docs")
workflow.add_edge("search_internal_docs", "decide_external")


def should_search_external(state: AgentState) -> str:
    """Route to external search or directly to generate response"""
    if state["needs_external_search"]:
        return "perform_external_search"

    return "generate_response"


workflow.add_conditional_edges(
    "decide_external",
    should_search_external,
    {
        "perform_external_search": "perform_external_search",
        "generate_response": "generate_response",
    },
)

workflow.add_edge("perform_external_search", "generate_response")
workflow.add_edge("generate_response", END)

compiled_workflow = workflow.compile()

In [101]:
try:
    # Generate the graph visualization
    png_data = compiled_workflow.get_graph().draw_mermaid_png()
    output_path = "svia_workflow.png"

    # Save to file
    with open(output_path, "wb") as f:
        f.write(png_data)

    print(f"📊 Workflow diagram saved to: {output_path}")
except Exception as e:
    print(f"⚠️  Could not generate workflow diagram: {e}")

📊 Workflow diagram saved to: svia_workflow.png


In [ ]:
query = "We are using Node.js with Express 4.17 for our API gateway. Are there known prototype pollution or remote code execution vulnerabilities?"

console.print(
    Panel.fit(f"🔐 [bold cyan]QUERY:[/bold cyan] {query}", border_style="cyan")
)

initial_state = AgentState(
    {
        "query": query,
    }
)

# Usar ainvoke ya que search_internal_docs es async
result = await compiled_workflow.ainvoke(initial_state)

console.print("\n")
console.print(
    Panel(
        Markdown(result["final_response"]),
        title="📋 FINAL RESPONSE",
        border_style="green",
        padding=(1, 2),
    )
)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔐 QUERY: We are using Node.js with Express 4.17 for our API gateway. Are there known prototype pollution or    │
│ remote code execution vulnerabilities?                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔍 Searching internal docs for: We are using Node.js with Express 4.17 for our API gateway. Are there known 
prototype pollution or remote code execution vulnerabilities?

✅ Agent response: There is a **critical** prototype pollution vulnerability in Node.js Express 4.17 that can lead 
to remote code execution (RCE). This vulnerability allows attackers to manipulate object prototypes thro...

🤔 External search needed: ✅ Yes

The internal information indicates a critical prototype pollution vulnerability in Express 4.17 that can result 
in remote code execution, and suggests upgrading to a newer version. However, it references findings from an 
internal report dated March 2024, which implies recent developments that might not yet be well-known or documented 
in public advisory databases due to the model's latest training cutoff being October 2023. Therefore, external 
searches in CVE databases and public security advisories could provide additional information about other potential
vulnerabilities, confirm the timeline, and offer further context or mitigation advice acknowledged by the wider 
security community. Searching external sources will ensure the most comprehensive understanding and response.

...

[0] https://nvd.nist.gov/vuln/detail/cve-2022-24999 - CVE-2022-24999 Detail - NVD

[1] https://www.usenix.org/system/files/sec23summer_432-shcherbakov-prepub.pdf - [PDF] Prototype Pollution Leads 
to Remote Code Execution in Node.js

[2] 
https://portswigger.net/web-security/prototype-pollution/server-side/lab-remote-code-execution-via-server-side-prot
otype-pollution - Lab: Remote code execution via server-side prototype pollution

🧠 Generating final correlated response...

╭─────────────────────────────────────────────── 📋 FINAL RESPONSE ───────────────────────────────────────────────╮
│                                                                                                                 │
│                                                Security Analysis                                                │
│                                                                                                                 │
│  Based on the provided information, it is evident that your current implementation using Node.js with Express   │
│  4.17 is highly vulnerable to a critical prototype pollution vulnerability, which can lead to remote code       │
│  execution (RCE). This vulnerability is documented both internally and externally, indicating a significant     │
│  threat to your systems.                                                                                        │
│                                                                                                                 │
│                                            Vulnerability Correlation                                            │
│                                                                                                                 │
│   1 Internal Documentation: There is a critical vulnerability in Express 4.17 allowing prototype pollution      │
│     that leads to RCE through manipulated JSON payloads in POST requests [internal knowledge].                  │
│   2 External Sources: This vulnerability aligns with CVE-2022-24999, highlighting the capability for            │
│     unauthenticated remote attackers to exploit unsafe input handling in Express [1]. Additionally,             │
│     proof-of-concept research corroborates the potential for RCE via prototype pollution in Node.js             │
│     applications, including Express [2]. Educational labs also demonstrate similar threats in default Express   │
│     configurations [3].                                                                                         │
│                                                                                                                 │
│                                               Severity and Impact                                               │
│                                                                                                                 │
│   • Severity: Critical. The vulnerability can be exploited remotely to execute arbitrary code, jeopardizing     │
│     both production and staging environments.                                                                   │
│   • Impact: Successful exploitation could lead to full system compromise, unauthorized access to sensitive      │
│     data, and potential disruption of services.                                                                 │
│                                                                                                                 │
│                                                Mitigation Steps                                                 │
│                                                                                                                 │
│   1 Immediate Upgrade:                                                                                          │
│      • Upgrade Express to version 4.18.2 or later to patch the known vulnerability [internal knowledge].        │
│      • Ensure all dependencies are updated to avoid additional vulnerabilities.                                 │
│   2 Implement Input Validation:                                                                                 │
│      • Use libraries such as Joi or express-validator to enforce strict schemas for user inputs, ensuring only  │
│        expected types and structures are processed.                                                             │
│      • Validate all incoming payloads, especially JSON,

In [72]:
es_client.indices.delete(index=INDEX_NAME)

ObjectApiResponse({'acknowledged': True})